In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp,year,month,dayofmonth
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor,DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [51]:
data = spark.read.csv("file:/home/hduser/hive/data/txns", header=False, inferSchema=True)\
.toDF("txnid","txndate","amount","custid","category","product","city","state","paymenttype")
data = data.withColumn("sales_date", unix_timestamp(col("txndate"), "MM-dd-yyyy").cast("timestamp"))
data = data.withColumn("year", year(col("sales_date")))
data = data.withColumn("month", month(col("sales_date")))
data = data.withColumn("day", dayofmonth(col("sales_date")))

indexers = [
    StringIndexer(inputCol="product", outputCol="product_index"),
    StringIndexer(inputCol="category", outputCol="category_index"),
    StringIndexer(inputCol="city", outputCol="city_index"),
    StringIndexer(inputCol="state", outputCol="state_index")
]
for indexer in indexers:
    data = indexer.fit(data).transform(data)
assembler = VectorAssembler(
    inputCols=["product_index", "category_index", "city_index", "state_index", "year","month","day"],
    outputCol="features"
)
data = assembler.transform(data)
final_data = data.select(col("features"), col("amount").alias("label"))

#Split Data
train_data, test_data = final_data.randomSplit([0.8, 0.2])

#Train Random Forest Model
rt_rf = RandomForestRegressor(featuresCol="features", labelCol="label", maxBins=320)
rf_model = rt_rf.fit(train_data)

rt_dt = DecisionTreeRegressor(featuresCol="features", labelCol="label", maxBins=320)
dt_model = rt_dt.fit(train_data)

rt_predictions = rf_model.transform(test_data)
rt_predictions.select("prediction", "label", "features")

dt_predictions = dt_model.transform(test_data)
dt_predictions.select("prediction", "label", "features")

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(rt_predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

rmse = evaluator.evaluate(dt_predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")



Root Mean Squared Error (RMSE) on test data = 2872.0285255542103
Root Mean Squared Error (RMSE) on test data = 2881.4910192736015


In [54]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


# Step 1: Load the data from a CSV file
file_path = "file:/home/hduser/insurance.csv"  # Path to the CSV file
data = spark.read.csv(file_path, header=True, inferSchema=True)
data.show(5)

# Step 2: Prepare the data
# Assemble features into a feature vector
feature_cols = ["age", "bmi", "children"]  # Modify according to your CSV's columns
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(data)

# Select only the necessary columns
data = data.select("features", "charges")

# Step 3: Train the Linear Regression model
# Initialize LinearRegression with the feature and label columns
lr = LinearRegression(featuresCol="features", labelCol="charges")

# Fit the model
lr_model = lr.fit(data)

# Print the coefficients and intercept
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

# Step 4: Save the trained model
model_path = "insurance_linear_regression_model"
lr_model.save(model_path)

# Step 5: Load the saved model and make predictions
from pyspark.ml.regression import LinearRegressionModel

# Load the model
loaded_model = LinearRegressionModel.load(model_path)

# Make predictions
predictions = loaded_model.transform(data)

# Select example rows to display
predictions.select("features", "charges", "prediction").show()

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="charges", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows

Coefficients: [239.99447429364724,332.0833645034438,542.8646522470166]
Intercept: -6916.243347786937
+-----------------+-----------+------------------+
|         features|    charges|        prediction|
+-----------------+-----------+------------------+
|  [19.0,27.9,0.0]|  16884.924| 6908.777533438443|
| [18.0,33.77,1.0]|  1725.5523| 9160.977061027028|
|  [28.0,33.0,3.0]|   4449.462|12390.946917789879|
|[33.0,22.705,0.0]|21984.47061| 8543.527094954

In [55]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



# Load the data
data = spark.read.csv("file:/home/hduser/insurance.csv", header=True, inferSchema=True)

# Display the schema of the DataFrame
data.printSchema()

# Data Preprocessing
# Index categorical columns
sex_indexer = StringIndexer(inputCol="sex", outputCol="sexIndexed")
smoker_indexer = StringIndexer(inputCol="smoker", outputCol="smokerIndexed")
region_indexer = StringIndexer(inputCol="region", outputCol="regionIndexed")

# Assemble features into a single vector
assembler = VectorAssembler(
    inputCols=["age", "sexIndexed", "bmi", "children", "smokerIndexed", "regionIndexed"],
    outputCol="features"
)

# Standardize the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

# Define the Linear Regression model
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="charges")

# Create the Pipeline
pipeline = Pipeline(stages=[sex_indexer, smoker_indexer, region_indexer, assembler, scaler, lr])

# Split the data into training and test sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=12345)

# Train the Pipeline
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="charges", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")

"""
Hyperparameter tuning is a crucial step in the machine learning workflow
to improve the performance of the model by finding the optimal combination of parameters.
"""

# Hyperparameter tuning using CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

# Train with CrossValidator
cvModel = crossval.fit(train_data)

# Make predictions with the best model
cvPredictions = cvModel.transform(test_data)

# Evaluate the best model
cvRmse = evaluator.evaluate(cvPredictions)
print(f"Cross-validated Root Mean Squared Error (RMSE) on test data: {cvRmse}")


root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)

Root Mean Squared Error (RMSE) on test data: 6248.945657672922
Cross-validated Root Mean Squared Error (RMSE) on test data: 6248.5170232884875
